Austin Placinto
I have neither given nor received any unauthorized aid in completing this work, nor have I presented someone else's work as my own.

In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.5'

In [0]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Initialize Directories

In [0]:
# Directories
train_dir =      '/gdrive/My Drive/vcc19sm2/train'
validation_dir = '/gdrive/My Drive/vcc19sm2/validation'
test_dir =       '/gdrive/My Drive/vcc19sm2/test'

train_circ_dir =      '/gdrive/My Drive/vcc19sm2/train/circle'
train_not_dir =      '/gdrive/My Drive/vcc19sm2/train/not_circle'
validation_circ_dir = '/gdrive/My Drive/vcc19sm2/validation/circle'
validation_not_dir = '/gdrive/My Drive/vcc19sm2/validation/not_circle'
test_circ_dir =       '/gdrive/My Drive/vcc19sm2/test/circle'
test_not_dir =       '/gdrive/My Drive/vcc19sm2/test/not_circle'

Make sure the data has been read.

In [0]:
import os, shutil
print('total training circle images:', len(os.listdir(train_circ_dir)))
print('total training non-circle images:', len(os.listdir(train_not_dir)))
print('total validation circle images:', len(os.listdir(validation_circ_dir)))
print('total validation non-circle images:', len(os.listdir(validation_not_dir)))
print('total test circle images:', len(os.listdir(test_circ_dir)))
print('total test non-circle images:', len(os.listdir(test_not_dir)))

Pre-processing.

In [0]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=0,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.1,
    zoom_range=0.2,
    #brightness_range=[1.2,1.5],
    horizontal_flip=True,)

train_generator = train_datagen.flow_from_directory(     
        train_dir, # This is the target directory
        target_size=(150, 150), # All images will be resized to 150x150
        batch_size=10,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=5,
        class_mode='binary')

Get shape

In [0]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

Build Model

In [0]:
from keras import layers
from keras import models
from keras import optimizers

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-3), # learning rate
              metrics=['acc'])

In [0]:
history = model.fit_generator(
      train_generator, # never see same image, for each epoch
      steps_per_epoch=100, # 1 x 60 samples for an epoch
      epochs=30,
      validation_data=validation_generator,
      validation_steps=50) # 1 (batch_size) x 15 = 15 validation samples

In [0]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [0]:
from keras.preprocessing.image import ImageDataGenerator
test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=1,
        class_mode='binary')

In [0]:
# To get the 1st batch
for t_data_batch, t_labels_batch in test_generator:
    print('data batch shape:', t_data_batch.shape)
    print('labels batch shape:', t_labels_batch.shape)
    break # try this for loop without "break"

In [0]:
import matplotlib.pyplot as plt
#plt.figure(figsize=(10,10))
for i in range(20):
  plt.subplot(4,5,i+1)
  plt.tight_layout()
  plt.imshow(t_data_batch[i],interpolation='none')
  plt.title("Label: {}".format(t_labels_batch[i]))
  plt.xticks([])
  plt.yticks([])

In [0]:
test_loss, test_acc = model.evaluate(test_generator)
print("Accuracy: ", test_acc)
print("Loss: ", test_loss)

The reason finding a high accuracy is difficult is because there are not enough samples to generate an effective model. I also believe that since the non_circle pictures are the ones that are set to true, the model cannot find a pattern for a "non-circle" since there are multiple shapes among those pictures. If the circle pictures were set to true instead I believe it would be able to recognize the patterns more easily.